In [1]:
from unityagents import UnityEnvironment
import numpy as np
env = UnityEnvironment(file_name="./VisualBanana_Linux/Banana.x86_64")

# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

UnityTimeOutException: The Unity environment took too long to respond. Make sure that :
	 The environment does not need user interaction to launch
	 The Academy and the External Brain(s) are attached to objects in the Scene
	 The environment and the Python interface have compatible versions.

In [ ]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents in the environment
print('Number of agents:', len(env_info.agents))

# number of actions
action_size = brain.vector_action_space_size
print('Number of actions:', action_size)

# examine the state space 
state = env_info.vector_observations[0]
print('States look like:', state)
state_size = len(state)
print('States have length:', state_size)

NameError: name 'env' is not defined

: 

In [ ]:
env_info = env.reset(train_mode=False)[brain_name] # reset the environment
state = env_info.vector_observations[0]            # get the current state
score = 0                                          # initialize the score
while True:
    action = np.random.randint(action_size)        # select an action
    env_info = env.step(action)[brain_name]        # send the action to the environment
    next_state = env_info.vector_observations[0]   # get the next state
    reward = env_info.rewards[0]                   # get the reward
    done = env_info.local_done[0]                  # see if episode has finished
    score += reward                                # update the score
    state = next_state                             # roll over the state to next time step
    if done:                                       # exit loop if episode finished
        break
    
print("Score: {}".format(score))


### Deep Q-Learning implementation

In [ ]:
env_info = env.reset(train_mode=False)[brain_name] # reset the environment
state = env_info.vector_observations[0]
score = 0    
for t in np.arange(0, 10):
    action = 1
    env_info = env.step(action)[brain_name]
    next_state = env_info.vector_observations[0]
    state = next_state
    print(state)


num_episodes = 1000
T = 20
# Initialize replay memory D with capacity N

# Initialize action-value function Q with weights theta^-

for i in range(num_episodes):
    # Initialize sequence 
    for t in range(T):
        # Agent act according to epsilon greedy with Q(s,a,theta)
        # action = ...
        # Execute action and observe reward and next state
        # Store experience in D
        # Sample random minibatch of transitions from D
        # Calculate target y = r_j + gamma * max ^Q (s', a; theta^-) (only r_j if done)
        # Update weights (gradient descent on squared error w.r.t parameters (y_j - Q(s,a,theta))
        # Every C steps reset ^Q = Q


In [ ]:
env.close()